In [1]:
import requests

In [20]:
import geopandas as gpd
import pandas as pd

In [10]:
from shapely.geometry import Point

## Open GIS Data

In [24]:
file_path = "data/cb_2018_us_state_500k/cb_2018_us_state_500k.shp"
states_df = gpd.read_file(file_path)

## Get State Population data

In [30]:
state_populations = pd.read_excel(
    "data/NST-EST2023-POP.xlsx", sheet_name=None, engine="openpyxl"
)

In [31]:
state_populations_df = state_populations["NST-EST2023-POP"][
    [
        "table with row headers in column A and column headers in rows 3 through 4. (leading dots indicate sub-parts)",
        "Unnamed: 5",
    ]
]
state_populations_df = state_populations_df.rename(
    columns={
        "table with row headers in column A and column headers in rows 3 through 4. (leading dots indicate sub-parts)": "NAME",
        "Unnamed: 5": "POPULATION",
    }
)
state_populations_df["NAME"] = state_populations_df["NAME"].str[1:]

In [27]:
states_with_population_df = states_df.merge(state_populations_df, on="NAME", how="left")
states_with_population_df = states_with_population_df[
    ["STUSPS", "POPULATION", "geometry"]
]

In [28]:
states_with_population_df = states_with_population_df.dropna()

## Get Lidl data

In [34]:
r = requests.get("https://mobileapi.lidl.com/v1/stores?")

In [35]:
stores_json = r.json()

In [38]:
store_dict = {"STUSPS": [], "geometry": []}
for store_json in stores_json["results"]:
    coords = store_json["coordinates"]
    p = Point(coords["lon"], coords["lat"])
    store_dict["STUSPS"].append(store_json["address"]["state"])
    store_dict["geometry"].append(p)

In [65]:
lidl_gdf = gpd.GeoDataFrame(store_dict, crs=4326)

In [66]:
totals = lidl_gdf.groupby("STUSPS").size()
totals_df = pd.DataFrame(
    {
        "STUSPS": totals.index,
        "COUNT": totals.values,
    }
)

## Merge data

In [53]:
states_with_population_with_totals_df = states_with_population_df.merge(
    totals_df, on="STUSPS", how="inner"
)

In [59]:
states_with_population_with_totals_df["people_per_store"] = (
    (
        states_with_population_with_totals_df["POPULATION"]
        / states_with_population_with_totals_df["COUNT"]
    )
    .round(decimals=0)
    .astype(int)
)
states_with_population_with_totals_df[
    "stores_per_1m"
] = states_with_population_with_totals_df["COUNT"] / (
    states_with_population_with_totals_df["POPULATION"] / 1_000_000
)

In [61]:
states_with_population_with_totals_df = states_with_population_with_totals_df[
    ["geometry", "COUNT", "people_per_store", "stores_per_1m"]
]

## Export

In [62]:
states_with_population_with_totals_df = states_with_population_with_totals_df.to_crs(
    32115
)

In [63]:
states_with_population_with_totals_df.to_file("data/states_with_counts.gpkg")

In [67]:
lidl_gdf = lidl_gdf.to_crs(32115)
lidl_gdf.to_file("data/stores.gpkg")